In [ ]:
# PBT

In [ ]:
! pip install -U tabulate

In [1]:
from ray import tune

In [ ]:
# problem example

In [2]:
import numpy as np
import argparse
import json
import os
import random

import ray
from ray.tune.schedulers import PopulationBasedTraining


def pbt_function(config, checkpoint_dir=None):
    """Toy PBT problem for benchmarking adaptive learning rate.

    The goal is to optimize this trainable's accuracy. The accuracy increases
    fastest at the optimal lr, which is a function of the current accuracy.

    The optimal lr schedule for this problem is the triangle wave as follows.
    Note that many lr schedules for real models also follow this shape:

     best lr
      ^
      |    /\
      |   /  \
      |  /    \
      | /      \
      ------------> accuracy

    In this problem, using PBT with a population of 2-4 is sufficient to
    roughly approximate this lr schedule. Higher population sizes will yield
    faster convergence. Training will not converge without PBT.
    """
    lr = config["lr"]
    accuracy = 0.0  # end = 1000
    start = 0
    if checkpoint_dir:
        with open(os.path.join(checkpoint_dir, "checkpoint")) as f:
            state = json.loads(f.read())
            accuracy = state["acc"]
            start = state["step"]

    midpoint = 100  # lr starts decreasing after acc > midpoint
    q_tolerance = 3  # penalize exceeding lr by more than this multiple
    noise_level = 2  # add gaussian noise to the acc increase
    # triangle wave:
    #  - start at 0.001 @ t=0,
    #  - peak at 0.01 @ t=midpoint,
    #  - end at 0.001 @ t=midpoint * 2,
    for step in range(start, 100):
        if accuracy < midpoint:
            optimal_lr = 0.01 * accuracy / midpoint
        else:
            optimal_lr = 0.01 - 0.01 * (accuracy - midpoint) / midpoint
        optimal_lr = min(0.01, max(0.001, optimal_lr))

        # compute accuracy increase
        q_err = max(lr, optimal_lr) / min(lr, optimal_lr)
        if q_err < q_tolerance:
            accuracy += (1.0 / q_err) * random.random()
        elif lr > optimal_lr:
            accuracy -= (q_err - q_tolerance) * random.random()
        accuracy += noise_level * np.random.normal()
        accuracy = max(0, accuracy)

        if step % 3 == 0:
            with tune.checkpoint_dir(step=step) as checkpoint_dir:
                path = os.path.join(checkpoint_dir, "checkpoint")
                with open(path, "w") as f:
                    f.write(json.dumps({"acc": accuracy, "step": start}))

        tune.report(
            mean_accuracy=accuracy,
            cur_lr=lr,
            optimal_lr=optimal_lr,  # for debugging
            q_err=q_err,  # for debugging
            done=accuracy > midpoint * 2  # this stops the training process
        )


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument(
        "--smoke-test", action="store_true", help="Finish quickly for testing")
    args, _ = parser.parse_known_args()
    if args.smoke_test:
        ray.init(num_cpus=2)  # force pausing to happen for test
    else:
        ray.init()

    pbt = PopulationBasedTraining(
        time_attr="training_iteration",
        perturbation_interval=4,
        hyperparam_mutations={
            # distribution for resampling
            "lr": lambda: random.uniform(0.0001, 0.02),
            # allow perturbations within this set of categorical values
            "some_other_factor": [1, 2],
        })

    analysis = tune.run(
        pbt_function,
        name="pbt_test",
        scheduler=pbt,
        verbose=False,
        metric="mean_accuracy",
        mode="max",
        stop={
            "training_iteration": 30,
        },
        num_samples=8,
        fail_fast=True,
        config={
            "lr": 0.0001,
            # note: this parameter is perturbed but has no effect on
            # the model training in this example
            "some_other_factor": 1,
        })

    print("Best hyperparameters found were: ", analysis.best_config)

2021-02-18 12:26:18,289	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265
2021-02-18 12:26:21,503	WARNING tune.py:411 -- Tune detects GPUs, but no trials are using GPUs. To enable trials to use GPUs, set tune.run(resources_per_trial={'gpu': 1}...) which allows Tune to expose 1 GPU to each trial. You can also override `Trainable.default_resource_request` if using the Trainable API.
2021-02-18 12:26:22,083	INFO pbt.py:535 -- [exploit] transferring weights from trial pbt_function_82205_00000 (score 4.364887976348562) -> pbt_function_82205_00005 (score 0.0)
2021-02-18 12:26:22,084	INFO pbt.py:550 -- [explore] perturbed config from {'lr': 0.0001, 'some_other_factor': 1} -> {'lr': 0.00012, 'some_other_factor': 1}
2021-02-18 12:26:22,089	INFO pbt.py:535 -- [exploit] transferring weights from trial pbt_function_82205_00000 (score 4.364887976348562) -> pbt_function_82205_00001 (score 0.0)
2021-02-18 12:26:22,089	INFO pbt.py:550 -- [explore] perturbed config from {'lr': 0.

2021-02-18 12:26:23,073	INFO pbt.py:535 -- [exploit] transferring weights from trial pbt_function_82205_00002 (score 19.06904521138547) -> pbt_function_82205_00001 (score 1.3641118535174495)
2021-02-18 12:26:23,073	INFO pbt.py:550 -- [explore] perturbed config from {'lr': 0.0001728, 'some_other_factor': 2} -> {'lr': 0.00020736, 'some_other_factor': 2}
(pid=2209) 2021-02-18 12:26:23,076	INFO trainable.py:372 -- Restored on 10.233.123.58 from checkpoint: /home/jovyan/ray_results/pbt_test/pbt_function_82205_00001_1_2021-02-18_12-26-21/checkpoint_tmp7d3182/./
(pid=2209) 2021-02-18 12:26:23,076	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 20, '_timesteps_total': None, '_time_total': 0.4714665412902832, '_episodes_total': None}
2021-02-18 12:26:23,138	INFO pbt.py:535 -- [exploit] transferring weights from trial pbt_function_82205_00002 (score 19.06904521138547) -> pbt_function_82205_00005 (score 4.300540779731486)
2021-02-18 12:26:23,138	INFO pbt.py:550 -- [explore]

Best hyperparameters found were:  {'lr': 0.0037287787435186844, 'some_other_factor': 1}


(pid=2220) 2021-02-18 12:26:23,706	INFO trainable.py:372 -- Restored on 10.233.123.58 from checkpoint: /home/jovyan/ray_results/pbt_test/pbt_function_82205_00004_4_2021-02-18_12-26-21/checkpoint_tmpea4f91/./
(pid=2220) 2021-02-18 12:26:23,706	INFO trainable.py:379 -- Current state after restoring: {'_iteration': 28, '_timesteps_total': None, '_time_total': 1.0364680290222168, '_episodes_total': None}
